<a href="https://colab.research.google.com/github/senemcet/automlcourse/blob/main/301125.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-optimize
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

data=load_breast_cancer()
X=data.data
y=data.target

In [ ]:
#pipeline ile uzay olusturma
pipeline=Pipeline([
    ('scaler',StandardScaler()),
    ('model',LogisticRegression(max_iter=2000))
])

#parametre dagilimi
search_spaces={
    "model__C": Real(1e-3, 1e3, prior="log-uniform"),
    "model__penalty":Categorical (["l2"]),
    "model__solver":Categorical(["lbfgs"])
}

In [ ]:
opt= BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces,
    n_iter=30,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
opt.fit(X,y)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                        ('model',
                                         LogisticRegression(max_iter=2000))]),
              n_iter=30, n_jobs=-1, random_state=42, scoring='accuracy',
              search_spaces={'model__C': Real(low=0.001, high=1000.0, prior='log-uniform', transform='normalize'),
                             'model__penalty': Categorical(categories=('l2',), prior=None),
                             'model__solver': Categorical(categories=('lbfgs',), prior=None)})

In [ ]:
import pandas as pd
results=pd.DataFrame(opt.cv_results_)
print(results[["params","mean_test_score","rank_test_score"]].sort_values(by="rank_test_score").head())

print("en iyi parametreler:",opt.best_params_)
print("en iyi skor:",opt.best_score_)
print("en iyi model:", opt.best_estimator_)

                                               params  mean_test_score  \
13  {'model__C': 0.7823086048734879, 'model__penal...         0.980686   
11  {'model__C': 0.8690904643609954, 'model__penal...         0.980686   
20  {'model__C': 0.854106496225252, 'model__penalt...         0.980686   
16  {'model__C': 0.9283242237084505, 'model__penal...         0.980686   
29  {'model__C': 0.5043678063439685, 'model__penal...         0.980686   

    rank_test_score  
13                1  
11                1  
20                1  
16                1  
29                1  
en iyi parametreler: OrderedDict({'model__C': 0.8690904643609954, 'model__penalty': 'l2', 'model__solver': 'lbfgs'})
en iyi skor: 0.9806862288464524
en iyi model: Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 LogisticRegression(C=0.8690904643609954, max_iter=2000))])


In [12]:
#Tpe

!pip install hyperopt

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

data= load_breast_cancer()
X=data.data
y=data.target


#arama uzayi olusturma

from hyperopt import hp,fmin,tpe,Trials

space={
    "C":hp.loguniform("C", np.log(1e-3), np.log(1e3)),
    "solver":hp.choice("solver",["lbfgs"]),
    "penalty":hp.choice("penalty",["l2"])
}


In [13]:
#nesne olusturma fonks

def objective(params):
  model=LogisticRegression(
      C=params["C"],
      solver=params["solver"],
      penalty=params["penalty"],
      max_iter=2000
  )

  scaler=StandardScaler()
  X_scaled=scaler.fit_transform(X)
  score=cross_val_score(model,X_scaled,y,cv=5,scoring="accuracy").mean()

  return -score  #cunku hyperopt minimize eder